In [46]:
!pip install selenium
!pip install bs4

In [47]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.by import By

In [48]:
main_link = 'https://www.tokopedia.com/search?st=&q=handphone&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource='

path = 'data\chrome.exe'

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.binary_location = path

driver = webdriver.Chrome(options=chrome_options)
time.sleep(10)
driver.get(main_link)

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6404\963250374.py:3: SyntaxWarning: invalid escape sequence '\c'
  path = 'data\chrome.exe'


In [49]:
html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")

In [50]:
shop_name, product_name, product_price, sold = [], [], [], []

def scroll_to_bottom(driver, scroll_pause_time=1, scroll_increment=1000):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script(f"window.scrollBy(0, {scroll_increment});")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def wait_for_elements(class_name, timeout=10):
    try:
        elements = WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, class_name))
        )
        return elements
    except:
        return []

def scrape_page():
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    product_elements = soup.find_all('div', class_='css-1nylpq2')
    if not product_elements:
        print("Product name not found.")
    else:
        for i in product_elements:
            product_name.append(i.text.strip() if i.text.strip() else "N/A")
            print(f"Product name: {i.text.strip()}")

    price_elements = soup.find_all('div', class_='price')
    if not price_elements:
        print("Product price not found.")
    else:
        for i in price_elements:
            product_price.append(i.text.strip() if i.text.strip() else "N/A")
            print(f"Product Price: {i.text.strip()}")

    shop_elements = soup.find_all('h2', class_='css-1wdzqxj-unf-heading e1qvo2ff2')
    if not shop_elements:
        print("Store name not found.")
    else:
        for i in shop_elements:
            shop_name.append(i.text.strip() if i.text.strip() else "N/A")
            print(f"Store name: {i.text.strip()}")

    sold_elements = soup.find_all('div', class_='items')
    if not sold_elements:
        print("The number of items sold was not found.")
    else:
        for i in sold_elements:
            sold.append(i.text.strip() if i.text.strip() else "N/A")
            print(f"Sold: {i.text.strip()}")

def click_next_page():
    try:
        next_button = driver.find_element(By.CLASS_NAME, 'css-bugrro-unf-pagination-item')
        if next_button:
            next_button.click()
            time.sleep(10)
            return True
    except:
        print("The next page button is not found or cannot be clicked.")
        return False

time.sleep(10)

menu_elements = wait_for_elements('css-5wh65g')

max_pages = 12  # Change this according to the desired number of pages
current_page = 1  # First page

while current_page <= max_pages:
    if not menu_elements:
        print("Menu not found, exit.")
        break
    else:
        total_elements = len(menu_elements)
        max_menu_elements = 85  # The maximum amount of data you want to process on each page
        print(f"Total menus found: {total_elements}, will process maximum {max_menu_elements} menu on the page {current_page}")

        for index in range(min(total_elements, max_menu_elements)):
            try:
                menu_elements = wait_for_elements('css-5wh65g')
                print(f"Clicking on the menu {index + 1} from maximum {max_menu_elements} menu")
                menu_elements[index].click()

                time.sleep(10)
                scrape_page()
                time.sleep(10)

                driver.back()
                time.sleep(10)

                menu_elements = wait_for_elements('css-5wh65g')
                
                scroll_to_bottom(driver, scroll_pause_time=3, scroll_increment=1000)

            except Exception as e:
                print(f"Error when clicking on element {index + 1}: {e}")
                break  

    if current_page >= max_pages or not click_next_page():
        print(f"There are no next pages or page limits ({max_pages}) has been achieved. Finished.")
        break

    current_page += 1

driver.quit()

Total menus found: 85, will process maximum 3 menu on the page 1
Clicking on the menu 1 from maximum 3 menu
Product name: Samsung Galaxy S24 FE [8/256] - [Blue, Graphite, Gray] Smartphone AI, AI Phone, Handphone AI, Kamera 50MP, Layar besar, RAM besar, Baterai tahan lama - Blue
Product Price: Rp9.999.000
Store name: Samsung Official Store
Sold: Terjual 50+•Diskusi (4)
Clicking on the menu 2 from maximum 3 menu
Product name: XIAOMI OFFICIAL Redmi Note 12 Pro 5G Kamera 50MP Sony IMX766 P-OLED - Sky Blue, 8/256GB
Product Price: Rp3.599.100
Store name: Xiaomi Official Store
Sold: Terjual 5 rb+•bintang 4.9 (2.827 rating)•Diskusi (414)
Clicking on the menu 3 from maximum 3 menu
Product name: Infinix Smart 8 Pro 8/128GB- Up to 16GB Extended RAM - 6.6" - Timber Black
Product Price: Rp1.369.000
Store name: Infinix Official Store
Sold: Terjual 3 rb+•bintang 4.9 (1.376 rating)•Diskusi (39)
Total menus found: 85, will process maximum 3 menu on the page 2
Clicking on the menu 1 from maximum 3 menu


In [51]:
min_length = min(len(product_name), len(product_price), len(shop_name), len(sold))

shop_name = shop_name[:min_length]
product_name = product_name[:min_length]
product_price = product_price[:min_length]
sold = sold[:min_length]

data = {
    'shop_name': shop_name,
    'product_name': product_name,
    'product_price': product_price,
    'sold': sold
}
df = pd.DataFrame(data)
df.to_csv('data/scraping_results.csv', index=False)

In [52]:
import pandas as pd
data = pd.read_csv("data/scraping_results.csv")
data

,shop_name,product_name,product_price,sold
0,Samsung Official Store,"Samsung Galaxy S24 FE [8/256] - [Blue, Graphit...",Rp9.999.000,Terjual 50+•Diskusi (4)
1,Xiaomi Official Store,XIAOMI OFFICIAL Redmi Note 12 Pro 5G Kamera 50...,Rp3.599.100,Terjual 5 rb+•bintang 4.9 (2.827 rating)•Disku...
2,Infinix Official Store,Infinix Smart 8 Pro 8/128GB- Up to 16GB Extend...,Rp1.369.000,Terjual 3 rb+•bintang 4.9 (1.376 rating)•Disku...
3,Samsung Official Store,"Samsung Galaxy S24 FE [8/256] - [Blue, Graphit...",Rp9.999.000,Terjual 50+•Diskusi (4)
4,Infinix Official Store,Infinix Smart 8 Pro 8/128GB- Up to 16GB Extend...,Rp1.369.000,Terjual 3 rb+•bintang 4.9 (1.376 rating)•Disku...
5,mnn gadget,GOOGLE PIXEL 6 PRO 5G - RAM 12GB INTERNAL 128G...,Rp3.500.000,Terjual 100+•bintang 4.8 (75 rating)•Diskusi (25)
6,Xiaomi Official Store,Official Xiaomi Redmi 13 (8GB/128GB) | (8GB/25...,Rp1.799.000,Terjual 1 rb+•bintang 5 (792 rating)•Diskusi (43)
7,Xiaomi Official Store,Official Xiaomi Redmi Note 13 Pro+ 5G | Layar ...,Rp5.899.000,Terjual 1 rb+•bintang 4.9 (862 rating)•Diskusi...
8,iSmile Official Store,Apple iPhone 15 Garansi Resmi - 128GB 256GB 51...,Rp13.529.000,Terjual 3 rb+•bintang 5 (1.783 rating)•Diskusi...
